##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.

import google.generativeai as genai

import base64
import json

try:
    # Mount google drive
    from google.colab import drive

    drive.mount("/gdrive")

    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = b'W3sicGFydHMiOiBbeyJ0ZXh0IjogIkhlbGxvIn1dfV0='
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

print(json.dumps(contents, indent=4))

Mounted at /gdrive
[
    {
        "parts": [
            {
                "text": "Hello"
            }
        ]
    }
]


## Call `generate_content`

In [4]:


# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash",
  generation_config=generation_config,
  system_instruction="Você vai receber um código em alguma linguagem de programação\nVocê deve analisar se é uma função válida.\nVocê deve analisar a função recebida e analisar se é uma função com múltiplas responsabilidades\nVocê deve indicar as responsabilidades executadas pela função , por exemplo:\n\nfuncao1 : possui as funcionalidades x, y, z\n\nEm seguida você deve Refatorar o código inicial criando as funções necessárias para manter o objetivo inicial da função problematica respeitando o principio da responsabilidade unica.",
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "def make_model(self, is_train: bool=False) -> tf.Tensor:\n        with tf.variable_scope(\"rnn_encoder\"):\n            self._make_placeholders()\n\n            self.placeholders['tokens_lengths'] = \\\n                tf.placeholder(tf.int32,\n                               shape=[None],\n                               name='tokens_lengths')\n\n            self.placeholders['rnn_dropout_keep_rate'] = \\\n                tf.placeholder(tf.float32,\n                               shape=[],\n                               name='rnn_dropout_keep_rate')\n\n            self.placeholders['rnn_recurrent_dropout_keep_rate'] = \\\n                tf.placeholder(tf.float32,\n                               shape=[],\n                               name='rnn_recurrent_dropout_keep_rate')\n\n            seq_tokens = self.placeholders['tokens']\n            seq_tokens_embeddings = self.embedding_layer(seq_tokens)\n            seq_tokens_lengths = self.placeholders['tokens_lengths']\n\n            rnn_final_state, token_embeddings = self._encode_with_rnn(seq_tokens_embeddings, seq_tokens_lengths)\n\n            output_pool_mode = self.get_hyper('rnn_pool_mode').lower()\n            if output_pool_mode == 'rnn_final':\n                return rnn_final_state\n            else:\n                token_mask = tf.expand_dims(tf.range(tf.shape(seq_tokens)[1]), axis=0)            # 1 x T\n                token_mask = tf.tile(token_mask, multiples=(tf.shape(seq_tokens_lengths)[0], 1))  # B x T\n                token_mask = tf.cast(token_mask < tf.expand_dims(seq_tokens_lengths, axis=-1),\n                                     dtype=tf.float32)                                            # B x T\n                return pool_sequence_embedding(output_pool_mode,\n                                               sequence_token_embeddings=token_embeddings,\n                                               sequence_lengths=seq_tokens_lengths,\n                                               sequence_token_masks=token_mask)",
      ],
    },
    {
      "role": "model",
      "parts": [
        "A função `make_model` possui as seguintes responsabilidades:\n\n1.  Cria placeholders para os dados de entrada (tokens, comprimentos dos tokens, dropout rates).\n2.  Realiza o embedding dos tokens de entrada.\n3.  Codifica a sequência de embeddings utilizando uma RNN.\n4.  Realiza o pooling da saída da RNN, utilizando diferentes modos (rnn\\_final, ou outros modos via `pool_sequence_embedding`).\n5.  Define o escopo das variáveis do TensorFlow como \"rnn\\_encoder\".\n6.  Retorna a saída da RNN ou o resultado do pooling, dependendo da configuração.\n\nO código não segue o princípio da responsabilidade única porque acumula várias responsabilidades distintas dentro de uma única função. Isso dificulta a leitura, o teste e a manutenção do código.\n\nAqui está uma refatoração que separa as responsabilidades em funções menores e mais focadas:\n\n```python\nimport tensorflow as tf\n\nclass ModelMaker:\n    def __init__(self, embedding_layer, get_hyper):\n        self.embedding_layer = embedding_layer\n        self.get_hyper = get_hyper\n        self.placeholders = {}\n\n    def make_model(self, is_train: bool=False) -> tf.Tensor:\n        \"\"\"Cria o modelo, orquestrando a criação de placeholders, embedding, codificação RNN e pooling.\"\"\"\n        with tf.variable_scope(\"rnn_encoder\"):\n            self._create_placeholders()\n\n            seq_tokens = self.placeholders['tokens']\n            seq_tokens_lengths = self.placeholders['tokens_lengths']\n\n            seq_tokens_embeddings = self._embed_tokens(seq_tokens)\n            rnn_final_state, token_embeddings = self._encode_with_rnn(seq_tokens_embeddings, seq_tokens_lengths)\n\n            return self._pool_rnn_output(rnn_final_state, token_embeddings, seq_tokens, seq_tokens_lengths)\n\n    def _create_placeholders(self):\n        \"\"\"Cria os placeholders necessários para o modelo.\"\"\"\n        self.placeholders['tokens'] = tf.placeholder(tf.int32, shape=[None, None], name='tokens') #Alterado para receber batches\n        self.placeholders['tokens_lengths'] = tf.placeholder(tf.int32, shape=[None], name='tokens_lengths')\n        self.placeholders['rnn_dropout_keep_rate'] = tf.placeholder(tf.float32, shape=[], name='rnn_dropout_keep_rate')\n        self.placeholders['rnn_recurrent_dropout_keep_rate'] = tf.placeholder(tf.float32, shape=[], name='rnn_recurrent_dropout_keep_rate')\n\n    def _embed_tokens(self, seq_tokens):\n        \"\"\"Realiza o embedding dos tokens de entrada.\"\"\"\n        return self.embedding_layer(seq_tokens)\n\n    def _encode_with_rnn(self, seq_tokens_embeddings, seq_tokens_lengths):\n        \"\"\"Codifica a sequência de embeddings utilizando uma RNN.\"\"\"\n        # Supondo que self._encode_with_rnn já existe e funciona como esperado\n        # Adapte conforme necessário\n        return self._encode_with_rnn(seq_tokens_embeddings, seq_tokens_lengths)\n\n    def _pool_rnn_output(self, rnn_final_state, token_embeddings, seq_tokens, seq_tokens_lengths):\n        \"\"\"Realiza o pooling da saída da RNN.\"\"\"\n        output_pool_mode = self.get_hyper('rnn_pool_mode').lower()\n        if output_pool_mode == 'rnn_final':\n            return rnn_final_state\n        else:\n            token_mask = self._create_token_mask(seq_tokens, seq_tokens_lengths)\n            return self._pool_sequence_embedding(output_pool_mode, token_embeddings, seq_tokens_lengths, token_mask)\n\n    def _create_token_mask(self, seq_tokens, seq_tokens_lengths):\n        \"\"\"Cria a máscara para os tokens.\"\"\"\n        token_mask = tf.expand_dims(tf.range(tf.shape(seq_tokens)[1]), axis=0)\n        token_mask = tf.tile(token_mask, multiples=(tf.shape(seq_tokens_lengths)[0], 1))\n        token_mask = tf.cast(token_mask < tf.expand_dims(seq_tokens_lengths, axis=-1), dtype=tf.float32)\n        return token_mask\n\n    def _pool_sequence_embedding(self, output_pool_mode, sequence_token_embeddings, sequence_lengths, sequence_token_masks):\n        \"\"\"Aplica a função de pooling na saída da RNN.\"\"\"\n        # Supondo que pool_sequence_embedding é uma função externa\n        return pool_sequence_embedding(output_pool_mode,\n                                       sequence_token_embeddings=sequence_token_embeddings,\n                                       sequence_lengths=sequence_lengths,\n                                       sequence_token_masks=sequence_token_masks)\n```\n\n**Mudanças:**\n\n*   **Criação de Classe:** Envolveu as funções em uma classe `ModelMaker` para encapsular o estado (placeholders, camadas, etc.) e facilitar a organização.\n*   **Funções Auxiliares:** Dividiu a função `make_model` em funções menores, cada uma com uma única responsabilidade:\n    *   `_create_placeholders`: Cria os placeholders.\n    *   `_embed_tokens`: Realiza o embedding dos tokens.\n    *   `_encode_with_rnn`: Codifica a sequência com a RNN (mantido como estava, assumindo que já está bem definido).\n    *   `_pool_rnn_output`: Lida com a lógica de pooling.\n    *   `_create_token_mask`: Cria a máscara para os tokens.\n    *   `_pool_sequence_embedding`: Aplica a função de pooling (mantido como uma chamada externa).\n*   **Orquestração:** A função `make_model` agora serve como um orquestrador, chamando as funções auxiliares na ordem correta.\n*   **Comentários:** Adicionados comentários para explicar o propósito de cada função.\n*   **Flexibilidade:**  A estrutura refatorada permite que você modifique ou substitua facilmente uma parte do processo (por exemplo, mudar a forma como o pooling é feito) sem afetar outras partes.\n\n**Benefícios:**\n\n*   **Legibilidade:** O código é mais fácil de entender, pois cada função tem um propósito claro.\n*   **Testabilidade:** Cada função pode ser testada individualmente.\n*   **Manutenibilidade:**  As mudanças em uma parte do código têm menos probabilidade de afetar outras partes.\n*   **Reusabilidade:** As funções auxiliares podem ser reutilizadas em outros lugares do código.\n\n**Observações:**\n\n*   Esta refatoração assume que `self._encode_with_rnn` e `pool_sequence_embedding` são funções definidas em outro lugar e que funcionam como esperado. Se essas funções também tiverem múltiplas responsabilidades, elas também devem ser refatoradas.\n*   A  função `_encode_with_rnn` foi mantida no exemplo por que foi citada no contexto, se ela possuir muitas responsabilidades deve ser refatorada seguindo o mesmo padrão.\n*   Adapte o código conforme necessário para o seu caso de uso específico.\n*   Considere usar type hints para tornar o código ainda mais claro.\n*   Lembre-se de que esta é apenas uma possível refatoração. Existem outras maneiras de dividir as responsabilidades, dependendo das suas necessidades.\n\nEste código está mais alinhado com o princípio da responsabilidade única e deve ser mais fácil de manter e testar.",
      ],
    },
  ]
)

response = chat_session.send_message("INSERT_INPUT_HERE")

print(response.text)

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

In [3]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>